In [1]:
import numpy as np
import h5py
import scipy.io
# np.random.seed(1337) # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD

from keras import initializations

def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.001, name=name)



Using Theano backend.


In [2]:
#Load data directly from file
f = open('goodmus.txt', 'r')
y = []
for line in f:
	y.append(float(line))

f = open('goodseqs.txt', 'r')
seqs = []
for line in f:
	seqs.append(line.strip('\n'))

# ATGC to one hot encoding
seq_map = {'A': np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'T': np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'G': np.array([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'C': np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
           'AA': np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'TA': np.array([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'GA': np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]), 
           'CA': np.array([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]),
           'AT': np.array([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]), 
           'TT': np.array([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]), 
           'GT': np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]), 
           'CT': np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]),
           'AC': np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]), 
           'TC': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]), 
           'GC': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]), 
           'CC': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]),
           'AG': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]), 
           'TG': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]), 
           'GG': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]), 
           'CG': np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
          }

#sequences to matrix form
X = []
for seq in seqs:
	X.append(np.vstack([seq_map[s] for s in seq]))
    
for i in range(len(X)):
    for j in range(len(seqs[i])-1):
        X[i]=np.vstack([X[i],seq_map[seqs[i][j]+seqs[i][j+1]]])

# zero pad sequences (end)
max_len = 0
for s in X:
	if len(s)>max_len:
		max_len = len(s)
for i in range(len(X)):
	p = max_len - X[i].shape[0]
	if p>0:
		X[i] = np.vstack([X[i], np.zeros((p,20))])

X= np.asarray(X)
y = np.asarray(y)
print X.shape


(28453, 225, 20)


In [3]:


#shuffle indices...not data!
idx = range(len(X))
np.random.shuffle(idx)
train_idx = idx[:int(len(X)*.85)]
test_idx = idx[int(len(X)*.85):]

#split into train/test
X_train = X[train_idx]
X_test = X[int(len(X)*.85):]
y_train = y[:int(len(X)*.85)]
y_test = y[int(len(X)*.85):]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], 1, X_train.shape[1])) #(N, F, H, W)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[2], 1, X_test.shape[1])) #(N, F, H, W)

print X_train.shape

(24185, 20, 1, 225)


In [9]:
lr = 1e-4#learning rate
reg = 1e-6

print 'building model'
model = Sequential()
model.add(Convolution2D(128, 4, 4, border_mode='same', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), W_regularizer=l2(reg), init=my_init))
model.add(Activation('relu'))
model.add(Convolution2D(256, 4, 4, border_mode='same', W_regularizer=l2(reg), init=my_init))
model.add(Activation('relu'))
model.add(Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(reg), init=my_init))
model.add(Activation('relu'))
model.add(Flatten())

model.add(Dense(1000))
model.add(Dense(1))
model.add(Activation('tanh')) #[-1, 1]


# In[17]:
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='mse',
              optimizer='adam', metrics=['mse'])
print 'compiling model'


building model
compiling model


In [10]:
# earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
# checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
# model.fit(X_train, y_train,nb_epoch=1000, batch_size=1024, verbose=1, show_accuracy=True, validation_data=(X_test, y_test),callbacks=[checkpointer,earlystopper])
model.fit(X_train, y_train,nb_epoch=1000, batch_size=512, verbose=1, show_accuracy=True, validation_data=(X_test, y_test))


/Users/jx/anaconda2/lib/python2.7/site-packages/Keras-1.1.1-py2.7.egg/keras/models.py:625: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 29803 samples, validate on 5260 samples
Epoch 1/1000
29803/29803 [==============================] - 999s - loss: 0.5821 - mean_squared_error: 0.5821 - val_loss: 0.6163 - val_mean_squared_error: 0.6163
Epoch 2/1000
29803/29803 [==============================] - 992s - loss: 0.6165 - mean_squared_error: 0.6165 - val_loss: 0.6163 - val_mean_squared_error: 0.6163
Epoch 3/1000
16384/29803 [===============>..............] - ETA: 428s - loss: 0.6154 - mean_squared_error: 0.6153

KeyboardInterrupt: 

In [ ]:
model.summary()
